In [1]:
# Installation step
%pip install numpy pandas bioverse==1.1.8


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Tutorial 1: Generating planetary systems (Data Saving Version)

This modified tutorial script runs the simulation steps and saves the resulting planetary sample to a CSV file for use as a test baseline.

## Setup

Let's start by importing the necessary modules.

In [2]:
# Import numpy, pandas, and os
import numpy as np
import pandas as pd
import os
import json

# Set a fixed random seed for reproducibility
np.random.seed(42)

# Import the Generator class and constants
from bioverse.generator import Generator
from bioverse.constants import ROOT_DIR

## Loading and Configuring the Generator

We load the `transit` generator and modify key arguments.

In [3]:
# Open the transit mode generator
generator = Generator('transit')

# Set eta_Earth = 15%
generator.set_arg('eta_Earth', 0.15)

# Set transit_mode = True
generator.set_arg('transit_mode', True)

val = str(generator.get_arg('transit_mode'))
print("\nThe current value of transit_mode is {:s}".format(val))


The current value of transit_mode is True


## Defining and Inserting a Custom Step

We define the `oceans` function and insert it into the generator.

In [4]:
# Define a function that accepts and returns a Table object
def oceans(d, f_ocean_min=0, f_ocean_max=1):
    # First, assign zero for all planets
    d['f_ocean'] = np.zeros(len(d))

    # Next, assign a random non-zero value for exo-Earth candidates
    EEC = d['EEC']
    d['f_ocean'][EEC] = np.random.uniform(f_ocean_min, f_ocean_max, size=EEC.sum())

    # Finally, return the Table with its new column
    return d

# Insert this function at the end of the Generator
generator.insert_step(oceans)

## Running the Generator and Saving Data

We run the generator and save the final sample data to a CSV file.

In [5]:
# Run the generator with all custom settings
sample = generator.generate(d_max=100, f_ocean_min=0.3, f_ocean_max=0.7)

print("Generated a sample of {:d} transiting planets including {:d} exo-Earth candidates.".format(len(sample), sample['EEC'].sum()))

# Convert the sample (Table) dictionary to a pandas DataFrame
df = pd.DataFrame(sample)

# Save the DataFrame to a CSV file for baseline testing
output_filename = 'tutorial_sample_data.csv'
df.to_csv(output_filename, index=False)

print(f"Final simulated sample data saved to {output_filename}")

Generated a sample of 16871 transiting planets including 1133 exo-Earth candidates.
Final simulated sample data saved to tutorial_sample_data.csv


## Clean Up

The remaining tutorial steps involve saving the Generator to disk and performing cleanup. We will execute the cleanup steps to ensure the test environment remains clean.

In [6]:
# First, save and reload the generator to simulate the full tutorial flow
generator.save('transit_oceans')
generator = Generator('transit_oceans')

# The file creation steps are skipped in this testing file, 
# but we include the file cleanup to ensure a clean state.

# Write a dummy function file to ensure cleanup runs, as per the original tutorial logic
func = """# dummy function
def oceans(d, f_ocean_min=0, f_ocean_max=1): return d"""
with open(ROOT_DIR+'/example_oceans.py', 'w') as f:
    f.write(func)

# Clean up files created during the exercise
trash = [ROOT_DIR+'/Objects/Generators/transit_oceans.pkl', ROOT_DIR+'/example_oceans.py']
for filename in trash:
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Cleaned up: {filename}")

Cleaned up: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bioverse/Objects/Generators/transit_oceans.pkl
Cleaned up: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bioverse/example_oceans.py
